<a href="https://colab.research.google.com/github/sugarforever/wtf-langchain/blob/main/05_Output_Parsers/05_Output_Parsers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05 输出解析器

LLM的输出为文本，但在程序中除了显示文本，可能希望获得更结构化的数据。这就是输出解析器（Output Parsers）的用武之地。

In [ ]:
pip install langchain==0.3.7
pip install langchain-chroma==0.1.4
pip install langchain-community==0.3.5
pip install langchain-core==0.3.18
pip install langchain-huggingface==0.1.2
pip install langchain-ollama==0.2.0
pip install langchain-openai==0.2.8
pip install langchain-text-splitters==0.3.2

## List Parser

List Parser将逗号分隔的文本解析为列表。

In [1]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("black, yellow, red, green, white, blue")

['black', 'yellow', 'red', 'green', 'white', 'blue']

## Structured Output Parser

当我们想要类似JSON数据结构，包含多个字段时，可以使用这个输出解析器。该解析器可以生成指令帮助LLM返回结构化数据文本，同时完成文本到结构化数据的解析工作。

In [2]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import OpenAI, ChatOpenAI
from langchain_ollama import OllamaLLM, ChatOllama

In [4]:
# 定义响应的结构(JSON)，两个字段 回答和 来源。
response_schemas = [
    ResponseSchema(name="回答", description="回答用户的问题"),
    ResponseSchema(name="来源", description="所提及的回答用户问题的来源，必须是一个网站。")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 获取响应格式化的指令
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"回答": string  // 回答用户的问题\n\t"来源": string  // 所提及的回答用户问题的来源，必须是一个网站。\n}\n```'

#### openai 聊天模型中使用

In [4]:
chat_model = ChatOpenAI(temperature=0, model_name="qwen2.5:14b", openai_api_key='随便写都可以', openai_api_base='http://localhost:11434/v1')

In [5]:
chat_prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("尽可能回答用户的问题。\n{format_instructions}\n{question}")  
    ],
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

_input = chat_prompt.format_prompt(question="what's the capital of france?")
print(_input)
output = chat_model.invoke(_input.to_messages())
output

messages=[HumanMessage(content='尽可能回答用户的问题。\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"回答": string  // 回答用户的问题\n\t"来源": string  // 所提及的回答用户问题的来源，必须是一个网站。\n}\n```\nwhat\'s the capital of france?', additional_kwargs={}, response_metadata={})]


AIMessage(content=' ```json\n{\n\t"回答": "The capital of France is Paris.",\n\t"来源": "https://www.britannica.com/place/Paris"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 112, 'total_tokens': 148, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5:14b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a544e16-42af-47ad-817f-16a632a7f96e-0', usage_metadata={'input_tokens': 112, 'output_tokens': 36, 'total_tokens': 148, 'input_token_details': {}, 'output_token_details': {}})

In [6]:
output_parser.parse(output.content)

{'回答': 'The capital of France is Paris.',
 '来源': 'https://www.britannica.com/place/Paris'}

In [7]:
_input = chat_prompt.format_prompt(question="Tesla的CEO是谁？")
output = chat_model.invoke(_input.to_messages())

In [8]:
output_parser.parse(output.content)

{'回答': '特斯拉（Tesla）的首席执行官是埃隆·马斯克（Elon Musk）。', '来源': 'https://www.tesla.com/'}

#### openai语言模型中使用

In [ ]:
这里使用 qwen 的模型会报错，但是使用 openai 的模型不会报错

In [ ]:
model = OpenAI(
    openai_api_base = 'http://localhost:11434/v1',
    openai_api_key = '随便写都可以',
    model_name = 'qwen2.5:14b'
)

In [5]:
prompt = PromptTemplate(
    template="尽可能回答用户的问题。\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
response = prompt.format_prompt(question="法国的首都是哪里？")
output = model.invoke(response.to_string())
output_parser.parse(output)

#### ollama 聊天模型使用

In [9]:
model = ChatOllama(temperature=0, model="qwen2.5:14b")
_input = chat_prompt.format_prompt(question="what's the capital of france?")
output = chat_model.invoke(_input.to_messages())
output

AIMessage(content=' ```json\n{\n\t"回答": "The capital of France is Paris.",\n\t"来源": "https://www.britannica.com/place/Paris"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 112, 'total_tokens': 148, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5:14b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-c9027597-d53d-46f4-8b4b-8278303628ba-0', usage_metadata={'input_tokens': 112, 'output_tokens': 36, 'total_tokens': 148, 'input_token_details': {}, 'output_token_details': {}})

In [10]:
output_parser.parse(output.content)

{'回答': 'The capital of France is Paris.',
 '来源': 'https://www.britannica.com/place/Paris'}

## 自定义输出解析器

扩展CommaSeparatedListOutputParser，让其返回的列表是经过排序的。

In [ ]:
from typing import List
class SortedCommaSeparatedListOutputParser(CommaSeparatedListOutputParser):
  def parse(self, text: str) -> List[str]:
    lst = super().parse(text)
    return sorted(lst)

output_parser = SortedCommaSeparatedListOutputParser()
output_parser.parse("black, yellow, red, green, white, blue")